### Transfer Recommendation Model

- This model will be used so players can select a player that is currently in their team adn the model can give them an alternative based off of statistics for the season
- My plan for this is to use a K Nearest Neighbours (KNN) model.
- I am planning to use this model as it will give me the nearest neighbour to the selected player and this will emulate the functionality that I need with the most similar player for the user to select.

In [1]:
import pandas as pd
import numpy as np
import sklearn

# Load the data
df = pd.read_csv('24_25_prem_player_stats.csv')

print(df.head())

  Rk               Player   Nation    Pos           Squad Age  Born  MP  \
0  1  Patrick van Aanholt   nl NED     DF  Crystal Palace  26  1990  28   
1  2       Rolando Aarons  eng ENG  MF,FW   Newcastle Utd  21  1995   4   
2  3        Tammy Abraham  eng ENG     FW    Swansea City  19  1997  31   
3  4         Charlie Adam  sct SCO     MF      Stoke City  31  1985  11   
4  5               Adrián   es ESP     GK        West Ham  30  1987  19   

  Starts   Min  ... Ast.1 G+A.1 G-PK.1 G+A-PK  xG.1 xAG.1 xG+xAG npxG.1  \
0     25  2184  ...  0.04  0.25   0.21   0.25  0.13  0.09   0.21   0.13   
1      1   139  ...  0.00  0.00   0.00   0.00  0.04  0.00   0.04   0.04   
2     15  1726  ...  0.05  0.31   0.26   0.31  0.35  0.08   0.44   0.35   
3      5   411  ...  0.00  0.00   0.00   0.00  0.36  0.27   0.63   0.19   
4     19  1710  ...  0.00  0.00   0.00   0.00  0.00  0.00   0.00   0.00   

  npxG+xAG.1  Matches  
0       0.21  Matches  
1       0.04  Matches  
2       0.44  Matches  
3 

In [11]:
# Ensure the first column is treated as a string and remove any leading/trailing spaces
df.iloc[:, 0] = df.iloc[:, 0].astype(str).str.strip()

# Filter out rows starting with 'Rk' (excluding the first row)
filtered_df = df.iloc[1:].copy()
filtered_df = filtered_df[~filtered_df.iloc[:, 0].str.startswith('Rk', na=False)]

# Re-add the first row (index 0) and reset index
df = pd.concat([df.iloc[[0]], filtered_df], ignore_index=True)

filtered_df = df.drop(columns=['Formula_Rating'])

# Save the cleaned data
filtered_df.to_csv('24_25_prem_player_stats_clean.csv', index=False)

df = pd.read_csv('24_25_prem_player_stats_clean.csv')

/var/folders/jp/fbwfkpy50vs_7m6dlkzl468h0000gn/T/ipykernel_90851/3661484786.py:2: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0        3
1        4
2        5
3        6
4        7
      ... 
522    525
523    526
524    527
525    528
526    529
Name: Rk, Length: 527, dtype: object' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.iloc[:, 0] = df.iloc[:, 0].astype(str).str.strip()


This block here is just removing some lines that were unneeded as they were just repeats of the header line. They were there as the table was split up into smalller blocks on the html page

In [14]:
Defender = pd.read_csv('defending/24_25_defending.csv')

# Removing all of the players that arent defenders
Defender = Defender[Defender['Pos'].str.startswith("DF", na=False)]

Defender.to_csv('24_25_defending_clean.csv', index=False)
Defender = pd.read_csv('24_25_defending_clean.csv')

# Drop all of the rows with null values
Defender.dropna()
df.dropna()

# Assigning the X values based on the position of the player
Attacker_X = df[['xG.1', 'Gls.1', 'npxG+xAG.1', 'xAG.1']]
Midfielder_X = df[['PrgP', 'xAG.1', 'xG.1', 'PrgC']]
Defender_X = Defender[['Tkl%', 'Tkl+Int', 'Clr', 'Blocks', 'Sh']]


Here I am reading in the defenders csv seperately as the statistics for the midfielders and attackers are all stored in my original dataset but my defender stats had to be gotten externally similarly to my previous model. The stats that I am going off of for this are the same ones that I used for my formula in my previous model, I am using thesde because they are based off of previous research that I have done from scouts and football statisticians.

My aim now is to train the KNN model with 1 neighbour to get a single player for them as a transfer recommendation. 

In [15]:
from sklearn.neighbors import NearestNeighbors

attacker_knn = NearestNeighbors(n_neighbors=1)
attacker_knn.fit(Attacker_X)

midfielder_knn = NearestNeighbors(n_neighbors=1)
midfielder_knn.fit(Midfielder_X)

defender_knn = NearestNeighbors(n_neighbors=1)
defender_knn.fit(Defender_X)

ValueError: Input X contains NaN.
NearestNeighbors does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values